
# Este notebook destina-se à formatação dos dados  utilizados no modelo

## Obtendo os dados não processados e o vetor de palavras pré-treinado(implementação utilizando o glove com 100 dimensões) do google drive

In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive', True)

In [0]:
cp "drive/My Drive/Colab Notebooks/PLN/text_sumarizer/data/news-of-the-site-folhauol.zip" "news-of-the-site-folhauol.zip"

In [0]:
cp "drive/My Drive/Colab Notebooks/PLN/text_sumarizer/models/glove_s100.zip" "glove_s100.zip"

In [0]:
cp -r "drive/My Drive/Colab Notebooks/PLN/nlputils" "nlputils"


In [0]:
cp "drive/My Drive/Colab Notebooks/PLN/text_sumarizer/requirements.txt" "requirements.txt"

In [0]:
from zipfile import ZipFile
with ZipFile("news-of-the-site-folhauol.zip", 'r') as zip_obj:
    zip_obj.extractall()
with ZipFile("glove_s100.zip", 'r') as zip_obj:
    zip_obj.extractall()

In [0]:
mv articles.csv dataset_artigos_de_noticias.csv

In [0]:
!pip3 install unidecode

     |████████████████████████████████| 245kB 3.5MB/s 


In [0]:
import nltk
nltk.download('punkt')
nltk.download('rslp')
nltk.download('stopwords')
nltk.download('perluniprops')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.


True

# Implementação do oprocessamento dos dados de um corpus de treinamento e de word embeddings pré-treinados

## Carregando o dataset

In [0]:
import pandas as pd

In [0]:
caminho_dataset = "dataset_artigos_de_noticias.csv"
dataset_noticias = pd.read_csv(caminho_dataset)

In [0]:
# Exibe as 5 primeiras linhas do dataset carregado
dataset_noticias.head()

,title,text,date,category,subcategory,link
0,"Lula diz que está 'lascado', mas que ainda tem...",Com a possibilidade de uma condenação impedir ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
1,"'Decidi ser escrava das mulheres que sofrem', ...","Para Oumou Sangaré, cantora e ativista malines...",2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
2,Três reportagens da Folha ganham Prêmio Petrob...,Três reportagens da Folha foram vencedoras do ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
3,Filme 'Star Wars: Os Últimos Jedi' ganha trail...,A Disney divulgou na noite desta segunda-feira...,2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
4,CBSS inicia acordos com fintechs e quer 30% do...,"O CBSS, banco da holding Elopar dos sócios Bra...",2017-09-10,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/10/1...


In [0]:
len(dataset_noticias)

167053

In [0]:
print("Título 3:\n", dataset_noticias['title'][3], '\n')
print("Texto 3:\n", dataset_noticias['text'][3], '\n')

Título 3:
 Filme 'Star Wars: Os Últimos Jedi' ganha trailer definitivo; assista 

Texto 3:
 A Disney divulgou na noite desta segunda-feira (9) o novo trailer de "Star Wars: Os Últimos Jedi", oitavo episódio da saga.  O trailer era aguardado pelos fãs e se tornou um dos tópicos mais comentados no Twitter no horário de seu lançamento.  Assista ao trailer de 'Os Últimos Jedi'  Assista ao trailer de 'Os Últimos Jedi'  Em "O Despertar da Força" (2015), episódio mais recente, a personagem Rey (Daisy Ridley) descobre que tem a Força e procura por Luke Skywalker (Mark Hamill) para começar seu treinamento Jedi.  A história do novo episódio continua desse ponto, e cenas do trailer mostram a relação entre Rey e Skywalker.  Com direção de Rian Johnson, o filme será lançado em 14 de dezembro no Brasil. O nono episódio, ainda sem título, encerra a nova trilogia em 20 de dezembro de 2019.  O estúdio também divulgou novo poster do filme.  Poster 



## Tokenizando por palavras

Obtendo um dicionário com todas as palavras distintas e suas respectivas frequências de ocorrência no dataset

In [0]:
from nlputils.lexical import normalization as norm
normalizer = norm.Normalizer()

Definindo uma rotina para processar o texto(remoção de informações inúteis)

In [0]:
stopwords = nltk.corpus.stopwords.words('portuguese')
nao_remover = ['não', 'muito', 'mais']
STOPWORDS = [word for word in stopwords if word not in nao_remover]

In [0]:
from nltk.tokenize.moses import MosesDetokenizer
detokenizer = MosesDetokenizer()

In [0]:
import re
def limpar_texto(texto, remover_stopwords=False):
    
    # Converte o texto para lower case
    texto = texto.lower()
    
    # Remove caracteres indesejados
    texto = re.sub(r'https?:\/\/.*[\r\n]*', '', texto, flags=re.MULTILINE)
    texto = re.sub(r'\<a href', ' ', texto)
    texto = re.sub(r'&amp;', '', texto) 
    texto = re.sub(r'[_"\%()|+&=*%!?:#$@\[\]/]', ' ', texto)
    texto = re.sub(r'<br />', ' ', texto)
    texto = re.sub(r'\'', ' ', texto)
    
    # Opcionalmente, remove stop words
    if remover_stopwords:
        tokens = [t for t in normalizer.tokenize_words(texto)
                    if t not in STOPWORDS]
        texto = detokenizer.detokenize(tokens, return_str=True)
        
    return texto
    

In [0]:
LIMITE = len(dataset_noticias)
titulos = []
textos = []
tokens_freqs = {} # As chaves serão as palavras e os valores serão as frequências de ocorrência
for i in range(LIMITE):
    titulo = limpar_texto(str(dataset_noticias['title'][i]))
    texto = limpar_texto(str(dataset_noticias['text'][i]), remover_stopwords=True)
    titulos.append(titulo)
    textos.append(texto)
    conteudo = f"{titulo}\n{texto}"
    tokens = normalizer.tokenize_words(conteudo)
    for token in tokens:
        if token in tokens_freqs:
            tokens_freqs[token] += 1
        else:
            tokens_freqs[token] = 1

Stop words foram removidas dos textos porque elas não provem muita utilidade no treinamento do modelo mas foram mantidas nos títulos(sumários) para que eles soem mais como frases naturais.

In [0]:
len(tokens_freqs)

490517

Ordenando os tokens por frequência de ocorrência

In [0]:
from collections import OrderedDict
from operator import itemgetter    

tokens_freqs = OrderedDict(sorted(tokens_freqs.items(), key = itemgetter(1), reverse = True))

In [0]:
print([(t, tokens_freqs[t]) for t in list(tokens_freqs)[:10]])

[(',', 5475607), ('.', 3343955), ('é', 617776), ('não', 616527), ('mais', 352199), ('ser', 171453), ('r', 148691), ('sobre', 147632), ('anos', 147300), ('disse', 138526)]


## Convertendo os tokens para representações vetoriais

Carregando os vetores de palavras pré-treinados(será usado o GloVe)

In [0]:
import numpy as np
# Dicionário para converter um token para vetor
token2vetor = {}

with open("glove_s100.txt", encoding="utf-8") as glove_file:
    for linha in glove_file: # Obs.: lixo será lido para a primeira linha
        valor = linha.split(' ')
        token = valor[0]
        vetor = np.asarray(valor[1:], dtype='float32')
        token2vetor[token] = vetor

In [0]:
token2vetor['de']

array([-0.059871, -0.26613 , -1.288952, -2.352894,  0.414237,  0.621193,
        1.073003, -0.548189, -0.643191,  0.966722,  0.869204, -0.212642,
       -1.038974, -0.356746, -0.24672 ,  0.471099,  0.049724, -0.7552  ,
        0.388665,  1.053411,  0.940066, -0.405713,  0.007156, -1.134779,
       -0.433221, -0.819711, -0.012   ,  0.032739, -0.225575,  1.153522,
        0.537173, -0.463806,  0.340472, -0.189793, -0.160288,  0.58365 ,
       -0.102506,  0.068884, -0.926536,  0.952415,  1.375001,  0.169173,
       -0.055722, -0.356511,  0.559938,  0.414697, -0.747112, -0.548186,
        0.35731 ,  0.851211,  0.036744, -0.746908, -0.305474, -0.137128,
       -1.137372, -0.514234, -0.272096, -0.622991, -0.762249, -0.625595,
        0.084059,  0.09621 ,  0.301824, -0.5573  ,  0.537335,  0.512624,
       -0.435838,  0.591526, -0.318885,  0.181027,  1.253085,  0.257935,
        0.734342,  0.245906,  2.948706,  0.0695  ,  0.484509, -0.255296,
        0.019143, -0.773821,  0.55352 , -1.709604, 

In [0]:
[token for token in list(token2vetor)[:10]]

['929605', ',', 'de', '.', 'a', 'o', 'e', 'que', 'do', 'da']

In [0]:
# Remove o lixo lido para a primeira linha
del token2vetor['929605']

# Confirmando a exclusão
[token for token in list(token2vetor)[:10]]

[',', 'de', '.', 'a', 'o', 'e', 'que', 'do', 'da', 'em']

Calculando o número de tokens do dataset que não não estão presentes no vocabulário do modelo GloVe e que têm uma frequência de ocorrência significativa

In [0]:
qtd_tokens_faltantes = 0
freq_min = 20

for token, freq in tokens_freqs.items():
    if freq > freq_min:
        if token.lower() not in token2vetor:
            qtd_tokens_faltantes += 1

percent_tok_falt = round(qtd_tokens_faltantes*100/len(tokens_freqs), 2)
print("Tokens faltantes no modelo GloVe: ", qtd_tokens_faltantes)
print(f"Percentual de tokens faltantes: {percent_tok_falt}%")

Tokens faltantes no modelo GloVe:  4711
Percentual de tokens faltantes: 0.96%


Foi usada um frequência mínima(freq_min) de ocorrência, para que tokens que não estejam presentes no modelo GloVe possam ser adicionados à word_embedding_matrix, mas eles precisam ocorrer o bastante nos textos, de forma que o modelo consiga entender seus significados

In [0]:
# Limita o vocabulário que será usado a tokens que têm uma 
# ocorrênica mínima >= freq_min ou fazem parte do vocabulário
# do modelo do glove

# Dicionário para converter tokens para inteiros
token2int = {}

# Dicionário para converter inteiros para tokens
int2token = {}

valor = 0
for token, freq in tokens_freqs.items():
    if freq >= freq_min or token.lower() in token2vetor:
        token2int[token] = valor
        int2token[valor] = token
        valor += 1

# Tokens especiais que serão adicionados ao dicionário
tokens_espec = ["<UNK>","<PAD>","<EOS>","<GO>"]

# Adiciona os tokens especiais ao vocabulário
for token in tokens_espec:
    valor = len(token2int) # Os códigos especias serão os maiores códigos
    token2int[token] = valor
    int2token[valor] = token

percent_uso_tok = round(len(token2int)/len(tokens_freqs), 4)*100    
    
print("Total de palavras distintas:", len(tokens_freqs))
print("Quantidade de palavras que serão usadas:", len(token2int))
print(f"Percentual de uso de tokens: {percent_uso_tok}%")

Total de palavras distintas: 490517
Quantidade de palavras que serão usadas: 269660
Percentual de uso de tokens: 54.97%


Construindo a matriz de tokens(embedding matrix)

In [0]:
import numpy as np

dim_vetor = 100 # Esse valor precisa conferir com o do modelo Glove
qtd_tokens = len(token2int)

# Cria a matrix com valores padrões como zero
matriz_de_tokens = np.zeros((qtd_tokens, dim_vetor), dtype=np.float32)

for token, valor in token2int.items():
    if token in token2vetor:
        matriz_de_tokens[valor] = token2vetor[token]
    else:
        # Se o token não estiver no modelo GloVe, cria um vetor aleatório para ele
        novo_vetor = np.array(np.random.uniform(-1.0, 1.0, dim_vetor))
        token2vetor[token] = novo_vetor
        matriz_de_tokens[valor] = novo_vetor
        
# Confere se o valor bate com len(token2int)
print(len(matriz_de_tokens))

269660


## Convertendo títulos e textos para inteiros

In [0]:

def converter_para_ints(dados_text, qtd_tokens, qtd_desc, eos=False):
    """
    Converte cada token no texto para um inteiro. Se o token
    não estiver em token2int, usa token2int('<UNK>')
    
    params: 
        dados_text:  uma lista de strings, o texto a ser convertido
        eos: bool, define se deve-se acrescentar ou não o 
             valor especial token2int('EOS') no final de cada 
             sentença
        qtd_tokens: int, uma quantidade de tokens anterior
        qtd_desc: int, um quantidade de tokens desconhecidos anterior
    return:
        ints: uma lista de inteiros, que representa o texto convertido
        qtd_tokens: int, a quantidade de tokens anterior mais os do texto
        qtd_desc: int, a quantidade de torkens desconhecidos anterior mais os do texto
    """
    ints = []
    for elemento in dados_text:
        ints_sent = []
        tokens = normalizer.tokenize_words(str(elemento))
        qtd_tokens += len(tokens)
        for token in tokens:
            if token.lower() in token2int:
                ints_sent.append(token2int[token.lower()])
            else:
                ints_sent.append(token2int['<UNK>'])
                qtd_desc += 1
        if eos:
            ints_sent.append(token2int['<EOS>'])
        ints.append(ints_sent)

    return ints, qtd_tokens, qtd_desc
            
            

In [0]:
# Aplica converter_para_ints a titulos e textos
qtd_tokens = 0
qtd_desc = 0

int_titulos, qtd_tokens, qtd_desc = converter_para_ints(titulos, qtd_tokens, qtd_desc)
int_textos, qtd_tokens, qtd_desc = converter_para_ints(textos, qtd_tokens, qtd_desc, eos=True)

percent_desc = round(qtd_desc*100/qtd_tokens, 2)

print("Número total de tokens:", qtd_tokens)
print("Número de tokens desconhecidos:", qtd_desc)
print(f"Percentual de tokens desconhecidos: {percent_desc}%")

Número total de tokens: 55873196
Número de tokens desconhecidos: 416224
Percentual de tokens desconhecidos: 0.74%


In [0]:
print(len(int_titulos), len(int_textos))
print("Representação textual:", titulos[3])
print("Representação inteira:", int_titulos[3])

167053 167053
Representação textual: filme  star wars  os últimos jedi  ganha trailer definitivo; assista
Representação inteira: [265, 4497, 6290, 2425, 236, 20884, 1783, 4849, 6014, 51, 4731]


Definindo um função que cria um dataframe de tamanhos de textos de uma lista de textos

In [0]:
def cria_tamanhos(lista_textos):
    tamanhos = []
    for texto in lista_textos:
        tamanhos.append(len(texto))
    return pd.DataFrame(tamanhos, columns=['quantidades'])

Criando dataframes com os tamanhos dos títulos e dos textos

In [0]:
tamanhos_titulos = cria_tamanhos(int_titulos)
tamanhos_textos = cria_tamanhos(int_textos)

print("Tamanho dos títulos:")
print(tamanhos_titulos.describe())
print()
print("Tamanhos dos textos:")
print(tamanhos_textos.describe())

Tamanho dos títulos:
         quantidades
count  167053.000000
mean       10.478190
std         3.069847
min         1.000000
25%         9.000000
50%        11.000000
75%        12.000000
max        27.000000

Tamanhos dos textos:
         quantidades
count  167053.000000
mean      324.985699
std       233.082603
min         1.000000
25%       183.000000
50%       287.000000
75%       412.000000
max      8732.000000


Definindo uma rotina para contar a quantidade de tokens desconhecidos em uma representação inteira de um texto qualquer

In [0]:
def conta_desc(int_texto):
    qtd_desc = 0
    for valor in int_texto:
        if valor == token2int["<UNK>"]:
            qtd_desc += 1
    return qtd_desc

Ordenando os títulos e textos por tamanho dos textos, do menor para o maior. Limitando o tamanho de títulos e textos baseado em valores min e max e removendo entradas que possuem muitos tokens desconhecidos.

In [0]:
# Dicionário para auxiliar na ordenação
# As chaves serão os tamanhos dos textos e os valores serão 
# tuplas da forma (int_titulo, int_texto)
conteudo_ordenado = {}

tam_min_texto = min(tamanhos_textos.quantidades)
tam_max_texto = max(tamanhos_textos.quantidades)
tam_min_titulo = min(tamanhos_titulos.quantidades)
tam_max_titulo = max(tamanhos_titulos.quantidades)
tam_min = 2
limite_desc_texto = 5
limite_desc_titulo = 1

limite = len(int_titulos)
for i in range(limite):
    tam_texto = len(int_textos[i])
    tam_titulo = len(int_titulos[i])
    qtd_desc_tit = conta_desc(int_titulos[i])
    qtd_desc_txt = conta_desc(int_textos[i])
    if tam_texto >= tam_min and tam_titulo >= tam_min and \
            qtd_desc_tit <= limite_desc_titulo and \
                qtd_desc_txt <= limite_desc_texto:
        conteudo_ordenado[len(int_textos[i])] = (int_titulos[i].copy(), int_textos[i].copy())

# Ordenando com base no tamanho dos textos
conteudo_ordenado = OrderedDict(sorted(conteudo_ordenado.items(), key = itemgetter(0)))

titulos_ordenados = []
textos_ordenados = []
for tam_texto, tup_tit_text in conteudo_ordenado.items():
    titulos_ordenados.append(tup_tit_text[0])
    textos_ordenados.append(tup_tit_text[1])

In [0]:
print(len(titulos_ordenados), len(textos_ordenados))
print(titulos_ordenados[0])
print(textos_ordenados[0])

1422 1422
[3484, 163, 27954, 30, 20, 167, 207, 475, 51, 455, 19334, 163, 11445]
[8153, 269658]


## Dados gerados

Os dados processados acima, foram salvos usando o pickle e podem ser encotrados em: [dados do modelo](https://drive.google.com/drive/folders/1b2bh5rYxr-RzJ8t2OiL3Cw0XqRNqGpTQ?usp=sharing)